<a href="https://colab.research.google.com/github/syjoe02/Krx-Bench/blob/main/KRX_Bench_synthetic_dataset_generation_with_raw_text_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KRX-Bench 합성 데이터셋 생성 with raw text 튜토리얼

- **litellm**은 다양한 LLM API를 OpenAI API로 통합하여 사용할 수 있는 라이브러리입니다.
- 본 튜토리얼에서는 OpenAI의 `gpt-4o-mini-2024-07-18` 모델을 활용하여 금융 관련 고품질 raw text 데이터셋 `alvanlii/finance-textbooks`을 QA Instruction 데이터셋으로 변환하는 방법에 대해 다룹니다.
- 데이터 생성 파이프라인:
 1. LLM에게 샘플링된 raw text를 입력으로 해서 질문 세트 생성
 2. 생성된 질문 세트에 대한 답변 생성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. litellm 설치 및 환경 설정
- 필요 라이브러리: pandas, datasets, random, litellm, os

In [ ]:
!pip install datasets litellm==1.44.9

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm

# OpenAI API key 선언
os.environ["OPENAI_API_KEY"] = "sk-xxx..."

## 2. 함수 선언

- 본 튜토리얼에서 사용하는 데이터셋인 `alvanlii/finance-textbooks`는 긴 길이의 텍스트 데이터로 구성되어 있기 때문에 적절한 길이의 데이터로 샘플링하는 작업이 필요합니다.
- `get_random_section`함수는 매우 긴 텍스트에서 무작위로 일부분을 샘플링하기 위한 함수로, 이 함수를 활용하면 긴 텍스트 데이터에서 적절한 길이의 텍스트 데이터를 샘플링해서 사용할 수 있습니다.
- 만약 `long_string`이 5000 글자 길이의 텍스트라면, 함수는 그 중 임의의 1000 글자 부분을 반환하게 됩니다.
- 이 기능은 임의로 텍스트 샘플을 추출하는 데 유용합니다.

In [ ]:
def get_random_section(long_string, length=1000):
    if len(long_string) <= length:
        return long_string

    start_index = random.randint(0, len(long_string) - length)

    return long_string[start_index:start_index + length]

## 3. 데이터셋 로드 및 전처리

- `alvanlii/finance-textbooks` 데이터셋은 금융 관련 고품질 raw text 데이터셋으로 instruction 형태가 아닌 raw text로 구성되어 있습니다.
- 데이터의 길이가 굉장히 길기 때문에 사전의 정의한 `get_random_section` 함수로 샘플링을 진행한 뒤 활용합니다.

In [ ]:
# 데이터셋 로드
ds = load_dataset('alvanlii/finance-textbooks')['train']

# 데이터셋 샘플링 - get_random_section
texts = []
for bt in ds['book_text']:
    for i in range(2):
        texts.append(get_random_section(bt, 2048))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/318 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.00M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60 [00:00<?, ? examples/s]

## 4. 합성 데이터셋 생성

합성 데이터셋 생성 파이프라인은 다음과 같습니다.
1. 샘플링된 raw text 데이터를 사용해서 질문 데이터셋을 생성합니다.
2. 생성된 질문 데이터셋에 대한 답변을 생성합니다.

In [ ]:
# chat prompt 포맷팅
qrys = []
for t in texts:
    messages = [
    # 첫번째 메세지 (role:system)는 모델에게 주어진 텍스트 기반으로 다중 단계를 거친 추론 질문을 만들으라고 지시
    {"content":"Your job is creating multi-hop reasoning questions in fluent Korean. You will be given a part of a text. Make a question based on it. The question should require multiple steps of reasoning related to the text. Return the question only without any other text.","role":"system"},
    # 두번째 메세지 (role:user)는 실제 텍스트(t)로, 이 텍스트를 기반으로 질문을 만들어야 한다
    { "content": t,"role": "user"}]
    qrys.append(messages)

# 1. raw text 데이터를 활용한 질문 생성
responses = batch_completion(
    model="gpt-4o-mini-2024-07-18",
    messages = qrys
)
# 각 응답의 첫번째 선택지 (choices)의 메세지 내용을 추출
resps = [i.choices[0].message.content for i in responses]
# 얼마나 많은 리소스를 사용했는지 측정
total_prompt_tokens_for_q = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_q = sum([r.usage.completion_tokens for r in responses])

# 최종적으로 텍스트와 생성된 질문을 dataframe으로 저장하여, 이를 분석하거나 다른용도로 사용할 수 있도록 정리
df = pd.DataFrame({'sampled_text':texts,'question':resps})


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Fee

AttributeError: 'AuthenticationError' object has no attribute 'choices'

In [ ]:
# 답변 생성용 prompt 포맷팅
qrys = []
for t in resps:
    messages = [
    # 모델에게 한국의 금융 전문가로서 질문에 답변하라고 지시
    {"content":"You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.","role":"system"},
    { "content": t,"role": "user"}]
    qrys.append(messages)

# 2. 생성된 질문에 대한 답변 생성
responses = batch_completion(
    model="gpt-4o-mini-2024-07-18",
    messages = qrys
)
resps = [i.choices[0].message.content for i in responses]
# df 데이터프레임에 생성된 답변을 새로운 열 response로 추가합니다
df['response'] = resps
# 모델의 리소스 사용량을 추적
total_prompt_tokens_for_a = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_a = sum([r.usage.completion_tokens for r in responses])

## 5. 데이터셋 생성 비용 확인

In [ ]:
print('total prompt tokens:', total_prompt_tokens_for_q + total_prompt_tokens_for_a)
print('prompt token costs: $', round((total_prompt_tokens_for_q + total_prompt_tokens_for_a) / 1000000 * 0.150, 6))
print('total completion tokens:', total_completion_tokens_for_q + total_completion_tokens_for_a)
print('completion token costs: $', round((total_completion_tokens_for_q + total_completion_tokens_for_a) / 1000000 * 0.600, 6))

## 6. 데이터셋 저장 및 확인

In [ ]:
# CSV 파일 저장
df.to_csv("output_path/result.csv")

# Excel 파일 저장
df.to_excel("output_path/result.xlsx")

# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
result_df = Dataset.from_pandas(df)
result_df.push_to_hub("hf/dataset", token="HF_TOKEN")

df.head()

## 참고자료

- [alvanlii/finance-textbooks](https://huggingface.co/datasets/alvanlii/finance-textbooks)
- [litellm Docs](https://docs.litellm.ai/)
- [Cosmopedia GitHub](https://github.com/huggingface/cosmopedia)
- [Cosmopedia Blog](https://huggingface.co/blog/cosmopedia)
- [Textbooks Are All You Need](https://arxiv.org/pdf/2306.11644)
- [Learning to Generate Instruction Tuning Datasets for Zero-Shot Task Adaptation](https://arxiv.org/abs/2402.18334)